#1) Introduction & Objectives

## What is RAG (Retrieval-Augmented Generation)?

**RAG** is a technique that combines **retrieval of relevant information** with **generation of text** using a large language model (LLM) like GPT. Instead of relying solely on the knowledge encoded in the model, RAG allows the model to **access external sources dynamically** to provide more accurate, up-to-date, or domain-specific answers.  

**How it works (high-level):**  
1. **Retrieve:** When a user asks a question, the system searches a **knowledge base** (e.g., documents, web pages) to find the most relevant chunks of information.  
2. **Augment:** These retrieved documents are added as **context** to the input of the language model.  
3. **Generate:** The LLM then uses both its internal knowledge and the retrieved context to produce a response.  

**Advantages of RAG:**  
- Reduces hallucinations (wrong answers) by grounding responses in real sources.  
- Keeps the model up-to-date without retraining.  
- Supports specialized domains like medical, legal, or academic knowledge.  

---

# Why combine OpenAI GPT with Pinecone vector DB for knowledge retrieval?

OpenAI GPT alone can generate fluent text, but it **cannot search through large external datasets efficiently**. To solve this:  

1. **Pinecone Vector Database** stores **embeddings** of documents (numerical representations of text).  
2. When a query is made, it is **converted into an embedding**.  
3. **Pinecone finds the most similar documents** using a similarity metric (e.g., cosine similarity).  
4. These retrieved documents are fed into GPT as **context**, enabling it to answer questions with **relevant external knowledge**.  

**Benefits of this combination:**  
- **Scalable retrieval:** Pinecone can store millions of documents efficiently.  
- **Dynamic knowledge:** GPT can provide natural language answers without needing to memorize every fact.  
- **Accurate responses:** Retrieval ensures the model’s answers are grounded in real data rather than only its training knowledge.  

---




# 2) Install Dependencies

In this step, we install all the Python libraries required to build our RAG Chatbot.  

### **Libraries and their purposes**

- **`langchain`** → A framework to build applications using Large Language Models (LLMs). It provides tools for chaining prompts, handling context, and integrating with external data sources.  

- **`langchain-openai`** → Provides connectors to OpenAI GPT models so that we can easily send prompts and receive responses from GPT.  

- **`langchain-pinecone`** → Integrates LangChain with the Pinecone vector database. It allows us to store, search, and retrieve vector embeddings efficiently for retrieval-augmented generation.  

- **`ipywidgets`** → Enables interactive user interfaces in Google Colab notebooks. We will use it to create input boxes, buttons, and display chat outputs dynamically.  

- **`tqdm`** → Provides progress bars for loops. This is helpful when embedding or indexing many documents so students can see progress in real-time.  

**Purpose:**  
Installing these packages ensures students understand not only **how to use them**, but also **why each is necessary** in a RAG workflow.  


In [8]:
!pip install -q langchain langchain-openai langchain-pinecone langchain-community pinecone-client tqdm ipywidgets

In [9]:
!pip install -qU \
    langchain==0.3.23 \
    langchain-community==0.3.21 \
    langchain-pinecone==0.2.5 \
    langchain-openai==0.3.12 \
    datasets==3.5.0

## 3) API Key Setup

In this step, we configure the API keys required to use OpenAI GPT models and Pinecone vector database.

### **Why API keys are needed**

- **OpenAI API key** → Allows access to OpenAI models (like GPT-4o-mini) to generate answers based on prompts.
- **Pinecone API key** → Provides access to Pinecone’s vector database for storing and retrieving embeddings of documents.

### **Security note**

- API keys are sensitive information.  
- We set them as **environment variables** (`os.environ`) so they are **not printed in the notebook output**.  
- This prevents accidental exposure if you share your notebook publicly.

### **How to use**

1. Replace the placeholder strings with your own API keys:
```python
OPENAI_API_KEY = "your_openai_api_key_here"
PINECONE_API_KEY = "your_pinecone_api_key_here"


In [10]:
import os
# === Edit these two variables only ===
OPENAI_API_KEY = "your_openai_api_key_here"
PINECONE_API_KEY = "your_pinecone_api_key_here"
# Set environment variables (not printed)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

print("✅ API keys configured (not displayed).")


✅ API keys configured (not displayed).


## 4) Helper Functions: Load, Chunk, Initialize, Create Knowledge Base

In this section, we define the **core helper functions** that are essential for building a RAG Chatbot. Each function has a specific role in preparing and indexing knowledge from web URLs.

### **1. `load_web_content(urls)`**
- Scrapes web pages from a list of URLs.
- Returns a list of **Document objects**, each containing the text content and metadata.
- Metadata includes the **source URL**, which is important for traceability. This helps later show **where each answer comes from**.

### **2. `chunk_documents(documents)`**
- Splits long text documents into smaller chunks.
- Why? LLMs like GPT have a **maximum input token limit**, and smaller chunks improve the quality of retrieval and relevance.
- Parameters:
  - `chunk_size` → maximum number of characters per chunk.
  - `chunk_overlap` → number of characters overlapping between chunks to preserve context.

### **3. `initialize_pinecone_index(api_key)`**
- Creates a **vector index** in Pinecone if it doesn’t exist.
- The index stores **document embeddings**, enabling efficient similarity search.
- Cosine similarity is used to find the most relevant chunks for a query.

### **4. `create_knowledge_base(urls, api_key)`**
- Combines all steps to create a retrievable knowledge base:
  1. Load web content using `load_web_content`.
  2. Split documents into chunks using `chunk_documents`.
  3. Generate embeddings for each chunk using **OpenAI embeddings**.
  4. Store embeddings in **Pinecone vector DB**.


In [11]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec, CloudProvider, AwsRegion, Metric
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.auto import tqdm

def load_web_content(urls):
    all_documents = []
    for url in urls:
        try:
            print(f"Loading content from: {url}")
            loader = WebBaseLoader(url)
            documents = loader.load()
            for doc in documents:
                doc.metadata['source'] = url
            all_documents.extend(documents)
            print(f"✓ Loaded {len(documents)} document(s) from {url}")
        except Exception as e:
            print(f"✗ Error loading {url}: {str(e)}")
    return all_documents

def chunk_documents(documents, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len)
    chunks = text_splitter.split_documents(documents)
    print(f"Split into {len(chunks)} chunks")
    return chunks

def initialize_pinecone_index(api_key, index_name="rag-pipeline"):
    pc = Pinecone(api_key=api_key)
    if not pc.has_index(name=index_name):
        pc.create_index(
            name=index_name,
            metric=Metric.COSINE,
            dimension=1536,
            spec=ServerlessSpec(cloud=CloudProvider.AWS, region=AwsRegion.US_EAST_1)
        )
        print(f"Created new index: {index_name}")
    else:
        print(f"Using existing index: {index_name}")
    return pc.Index(name=index_name)

def create_knowledge_base(urls, api_key, index_name="rag-pipeline"):
    documents = load_web_content(urls)
    if not documents:
        raise ValueError("No documents were loaded successfully")
    chunks = chunk_documents(documents)
    embed_model = OpenAIEmbeddings(model="text-embedding-3-small")
    index = initialize_pinecone_index(api_key, index_name)
    print("Embedding and indexing documents...")
    batch_size = 100
    for i in tqdm(range(0, len(chunks), batch_size)):
        i_end = min(len(chunks), i + batch_size)
        batch = chunks[i:i_end]
        ids = [f"doc-{i+j}" for j in range(len(batch))]
        texts = [doc.page_content for doc in batch]
        embeds = embed_model.embed_documents(texts)
        metadata = [{'text': doc.page_content, 'source': doc.metadata.get('source', 'unknown')} for doc in batch]
        index.upsert(vectors=zip(ids, embeds, metadata))
    print(f"✓ Indexed {len(chunks)} chunks")
    return PineconeVectorStore(index=index, embedding=embed_model, text_key="text")

def augment_prompt(query, vectorstore, k=3):
    results = vectorstore.similarity_search(query, k=k)
    source_knowledge = "\n\n".join([x.page_content for x in results])
    sources = [x.metadata.get('source', 'unknown') for x in results]
    augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""
    return augmented_prompt, sources


## 5) Create Knowledge Base (Run Once)

In this step, we **load, embed, and index all web pages** into Pinecone to create a knowledge base.  

### **Key Points for Students**

- **Indexing is expensive:**  
  - Embedding each chunk of text requires API calls to OpenAI.  
  - Uploading embeddings to Pinecone also takes time.  
  - Therefore, we **run this step only once**. After the knowledge base is created, we can reuse it for multiple queries.

- **Vectorstore creation:**  
  - The `create_knowledge_base(urls, api_key)` function returns a **vectorstore object**, which contains the embeddings and Pinecone index.  
  - This is stored in a variable named `vectorstore` for reuse.  
  - Example:  
    ```python
    vectorstore = create_knowledge_base(urls, api_key=PINECONE_API_KEY, index_name="rag-pipeline")
    ```
  - Once created, you can use `vectorstore` for **retrieval without re-indexing**.

- **Editing URLs:**  
  - Students can **add or remove web pages** in the `urls` list to change the knowledge base content.  
  - Example:  
    ```python
    urls = [
        "https://www.nwmissouri.edu/csis/msacs/",
        "https://www.nwmissouri.edu/csis/msacs/about.htm",
        # Add your own URLs here
    ]
    ```

**Purpose:**  
This step teaches students how **raw web content is converted into a retrievable knowledge base** and emphasizes the cost and time implications of embedding and indexing.


In [12]:
# Example URLs (edit if needed)
urls = [
    # Depth 1 URLs
    "https://www.nwmissouri.edu/giving/foundation/board/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/pre-osteopathy.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/political-science.htm",
    "https://www.nwmissouri.edu/studentaccounts/tuitionandfees.htm",
    "https://www.nwmissouri.edu/alumni/missing/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/pre-respiratory-therapy.htm",
    "https://www.nwmissouri.edu/wellness/Hope4All/index.htm",
    "https://www.nwmissouri.edu/hr/tobaccofree/index.htm",
    "https://www.nwmissouri.edu/socialsciences/index.htm",
    "https://www.nwmissouri.edu/recreation/SRC/index.htm",
    "https://www.nwmissouri.edu/hr/index.htm",
    "https://www.nwmissouri.edu/recreation/HughesFieldhouse/index.htm",
    "https://www.nwmissouri.edu/math/mathlab.htm",
    "https://www.nwmissouri.edu/housing/current/get-involved/index.htm",
    "https://www.nwmissouri.edu/tickets/index.htm",
    "https://www.nwmissouri.edu/naturalsciences/sds/index.htm",
    "https://www.nwmissouri.edu/international/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/international-business.htm",
    "https://www.nwmissouri.edu/alumni/association/benefits.htm",
    "https://www.nwmissouri.edu/aboutus/facts/consumerinfo.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/middle-school-education.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/applied-health-science.htm",
    "https://www.nwmissouri.edu/academics/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/recovery/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/art-studio.htm",
    "https://www.nwmissouri.edu/ag/about/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/support/tutoring.htm",
    "https://www.nwmissouri.edu/compserv/Laptops/index.htm",
    "https://www.nwmissouri.edu/fsenate/irb/index.htm",
    "https://www.nwmissouri.edu/getinvolved/index.htm",
    "https://www.nwmissouri.edu/wellness/index.htm",
    "https://www.nwmissouri.edu/graduate/index.htm",
    "https://www.nwmissouri.edu/services/facility/index.htm",
    "https://www.nwmissouri.edu/recreation/index.htm",
    "https://www.nwmissouri.edu/compserv/about-us.htm",
    "https://www.nwmissouri.edu/schedule/index.htm",
    "https://www.nwmissouri.edu/fsl/phc/sk.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/finance.htm",
    "https://www.nwmissouri.edu/services/ir/index.htm",
    "https://www.nwmissouri.edu/fsl/phc/adp.htm",
    "https://www.nwmissouri.edu/alumni/events/index.htm",
    "https://www.nwmissouri.edu/studentaffairs/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/ag-education.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/theatre.htm",
    "https://www.nwmissouri.edu/academics/schools.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/graphic-design.htm",
    "https://www.nwmissouri.edu/communications/programs/index.htm",
    "https://www.nwmissouri.edu/provost/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/firstgen/index.htm",
    "https://www.nwmissouri.edu/fsenate/index.htm",
    "https://www.nwmissouri.edu/aboutus/regents/index.htm",
    "https://www.nwmissouri.edu/rpdc/index.htm",
    "https://www.nwmissouri.edu/business/index.htm",
    "https://www.nwmissouri.edu/aboutus/comment.htm",
    "https://www.nwmissouri.edu/admissions/campusvisits/maps.htm",
    "https://www.nwmissouri.edu/login/index.htm",
    "https://www.nwmissouri.edu/admissions/campusvisits/index.htm",
    "https://www.nwmissouri.edu/parents/index.htm",
    "https://www.nwmissouri.edu/alumni/travel/index.htm",
    "https://www.nwmissouri.edu/marketing/multimedia/photography.htm",
    "https://www.nwmissouri.edu/recreation/intramurals/index.htm",
    "https://www.nwmissouri.edu/health/programs/index.htm",
    "https://www.nwmissouri.edu/graduation/index.htm",
    "https://www.nwmissouri.edu/academics/majors.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/chemistry-education.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/earth-science-education.htm",
    "https://www.nwmissouri.edu/assessment/tests.htm",
    "https://www.nwmissouri.edu/communications/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/advising/4yearplan.htm",
    "https://www.nwmissouri.edu/horacemann/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/advertising.htm",
    "https://www.nwmissouri.edu/admissions/campusvisits/lodging.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/biology-biomedical-science.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/english-education.htm",
    "https://www.nwmissouri.edu/registrar/finals.htm",
    "https://www.nwmissouri.edu/studentaffairs/foodpantry/index.htm",
    "https://www.nwmissouri.edu/academics/majors.htm?search=geographic%20information",
    "https://www.nwmissouri.edu/math/index.htm",
    "https://www.nwmissouri.edu/academics/ltc/index.htm",
    "https://www.nwmissouri.edu/aboutus/history.htm",
    "https://www.nwmissouri.edu/compserv/technology-policies.htm",
    "https://www.nwmissouri.edu/ag/programs/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/music.htm",
    "https://www.nwmissouri.edu/textbooks/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/chemistry.htm",
    "https://www.nwmissouri.edu/titleixequity/accessibility/index.htm",
    "https://www.nwmissouri.edu/education/peu/certification/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/mathematics.htm",
    "https://www.nwmissouri.edu/getinvolved/sac/index.htm",
    "https://www.nwmissouri.edu/compserv/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/pre-radiology.htm",
    "https://www.nwmissouri.edu/fsl/ifc/spe.htm",
    "https://www.nwmissouri.edu/services/sustainability//index.htm",
    "https://www.nwmissouri.edu/housing/index.htm",
    "https://www.nwmissouri.edu/wellness/education/peered/index.htm",
    "https://www.nwmissouri.edu/studentaffairs/auxiliary/card.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/accounting.htm",
    "https://www.nwmissouri.edu/fsl/phc/pm.htm",
    "https://www.nwmissouri.edu/admissions/apply/index.htm",
    "https://www.nwmissouri.edu/naturalsciences/index.htm",
    "https://www.nwmissouri.edu/education/peu/tess/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/ag-business.htm",
    "https://www.nwmissouri.edu/familyweekend/index.htm",
    "https://www.nwmissouri.edu/fsl/index.htm",
    "https://www.nwmissouri.edu/marketing/index.htm",
    "https://www.nwmissouri.edu/housing/dining/index.htm",
    "https://www.nwmissouri.edu/titleixequity/BearcatEquity/index.htm",
    "https://www.nwmissouri.edu/academics/dualcredit/index.htm",
    "https://www.nwmissouri.edu/ag/index.htm",
    "https://www.nwmissouri.edu/conferences/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/marketing.htm",
    "https://www.nwmissouri.edu/studentaccounts/tuition.htm",
    "https://www.nwmissouri.edu/policies/index.htm",
    "https://www.nwmissouri.edu/education/peu/studentteach/index.htm",
    "https://www.nwmissouri.edu/assessment/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/pre-dental-hygiene.htm",
    "https://www.nwmissouri.edu/trio/ubms/index.htm",
    "https://www.nwmissouri.edu/academics/catalog/index.htm",
    "https://www.nwmissouri.edu",
    "https://www.nwmissouri.edu/academics/graduate/masters/business-administration.htm",
    "https://www.nwmissouri.edu/marketing/multimedia/index.htm",
    "https://www.nwmissouri.edu/academics/grants/index.htm",
    "https://www.nwmissouri.edu/education/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/physical-education.htm",
    "https://www.nwmissouri.edu/studentaffairs/payitforward/index.htm",
    "https://www.nwmissouri.edu/trio/sss/index.htm",
    "https://www.nwmissouri.edu/llw/writingcenter/index.htm",
    "https://www.nwmissouri.edu/library/index.htm",
    "https://www.nwmissouri.edu/aboutus/facts/index.htm",
    "https://www.nwmissouri.edu/alumni/index.htm",
    "https://www.nwmissouri.edu/admissions/contact.htm",
    "https://www.nwmissouri.edu/academics/celebration/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/geography.htm",
    "https://www.nwmissouri.edu/fsl/phc/asa.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/management-information-systems.htm",
    "https://www.nwmissouri.edu/giving/directory/index.htm",
    "https://www.nwmissouri.edu/international/studyabroad/index.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/concertchoir.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/pre-pharmacy.htm",
    "https://www.nwmissouri.edu/veterans/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/petitions/index.htm",
    "https://www.nwmissouri.edu/virtualtour/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/advising/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/foods-and-nutrition.htm",
    "https://www.nwmissouri.edu/map/parking.htm",
    "https://www.nwmissouri.edu/health/index.htm",
    "https://www.nwmissouri.edu/studentaffairs/ssenate/index.htm",
    "https://www.nwmissouri.edu/housing/current/handbook/my-room.htm",
    "https://www.nwmissouri.edu/fsl/ifc/pdt.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/history.htm",
    "https://www.nwmissouri.edu/archives/index.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/marching/index.htm",
    "https://www.nwmissouri.edu/police/programs/saferide.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/computer-science.htm",
    "https://www.nwmissouri.edu/llw/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/support/si.htm",
    "https://www.nwmissouri.edu/fsl/ifc/akl.htm",
    "https://www.nwmissouri.edu/giving/index.htm",
    "https://www.nwmissouri.edu/llw/programs/index.htm",
    "https://www.nwmissouri.edu/media/rss.xml",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/writing.htm",
    "https://www.nwmissouri.edu/academics/honors/index.htm",
    "https://www.nwmissouri.edu/maps/index.htm",
    "https://www.nwmissouri.edu/aboutus/president/index.htm",
    "https://www.nwmissouri.edu/events/index.htm",
    "https://www.nwmissouri.edu/alert/index.htm",
    "https://www.nwmissouri.edu/recreation/MOERA/index.htm",
    "https://www.nwmissouri.edu/hr/student/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/business-education.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/index.htm",
    "https://www.nwmissouri.edu/admissions/index.htm",
    "https://www.nwmissouri.edu/academics/graduate/masters/info-systems.htm",
    "https://www.nwmissouri.edu/academics/calendar.htm",
    "https://www.nwmissouri.edu/wellness/engage/index.htm",
    "https://www.nwmissouri.edu/connection/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/digital-media.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/clinical-laboratory-science.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/animal-science.htm",
    "https://www.nwmissouri.edu/arboretum/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/art-education.htm",
    "https://www.nwmissouri.edu/homecoming/index.htm",
    "https://www.nwmissouri.edu/archives/computing/index.htm",
    "https://www.nwmissouri.edu/recreation/FitnessCenter/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/communication.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/pre-law.htm",
    "https://www.nwmissouri.edu/studentaffairs/auxiliary/index.htm",
    "https://www.nwmissouri.edu/finaid/aid/scholarships/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/speech-and-theatre-education.htm",
    "https://www.nwmissouri.edu/directory.htm",
    "https://www.nwmissouri.edu/library",
    "https://www.nwmissouri.edu/esports/index.htm",
    "https://www.nwmissouri.edu/trio/index.htm",
    "https://www.nwmissouri.edu/trio/upwardbound/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/music-education.htm",
    "https://www.nwmissouri.edu/studentaccounts/index.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/tower/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/agricultural-media.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/pre-nursing.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/mathematics-education.htm",
    "https://www.nwmissouri.edu/services/purchasing/index.htm",
    "https://www.nwmissouri.edu/naturalsciences/programs/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/geology.htm",
    "https://www.nwmissouri.edu/alumni/association/index.htm",
    "https://www.nwmissouri.edu/academics/dontcancel/index.htm",
    "https://www.nwmissouri.edu/education/programs/index.htm",
    "https://www.nwmissouri.edu/registrar/transcript.htm",
    "https://www.nwmissouri.edu/intranet/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/nanoscale-science.htm",
    "https://www.nwmissouri.edu/fsl/phc/sss.htm",
    "https://www.nwmissouri.edu/extendedlearning/index.htm",
    "https://www.nwmissouri.edu/alumni/events/awards/index.htm",
    "https://www.nwmissouri.edu/media/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/pre-engineering.htm",
    "https://www.nwmissouri.edu/registrar/index.htm",
    "https://www.nwmissouri.edu/current.htm",
    "https://www.nwmissouri.edu/giving/ways-to-give/onlinegiving.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/spanish.htm",
    "https://www.nwmissouri.edu/alumni/chapters/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/index.htm",
    "https://www.nwmissouri.edu/housing/current/get-involved/rha/index.htm",
    "https://www.nwmissouri.edu/fsl/ifc/agr.htm",
    "https://www.nwmissouri.edu/police/index.htm",
    "https://www.nwmissouri.edu/csis/index.htm",
    "https://www.nwmissouri.edu/aboutus/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/economics.htm",
    "https://www.nwmissouri.edu/studentaffairs/notary/index.htm",
    "https://www.nwmissouri.edu/alumni/magazine/index.htm",
    "https://www.nwmissouri.edu/studentaffairs/bit.htm",
    "https://www.nwmissouri.edu/finearts/index.htm",
    "https://www.nwmissouri.edu/finaid/index.htm",
    "https://www.nwmissouri.edu/career/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/business-management.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/agronomy.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/mass-media.htm",

    # Depth 2 URLs
    "https://www.nwmissouri.edu/giving/foundation/index.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/gruhn.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/vansaghi.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/pijanowski.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/moskow.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/hockensmith.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/shaffer.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/hewitt.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/ginasmith.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/pfost.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/athen.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/Burkhead.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/roe.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/gaarder.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/strider.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/murphy.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/tunell.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/tplace.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/sullivan.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/griffin.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/powell.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/greene.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/fullerton.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/dstephens.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/tokheim.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/headrick.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/mikekelly.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/Clarance.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/edge.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/barnes.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/baumgartner.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/burns.htm",
    "https://www.nwmissouri.edu/giving/foundation/board/jimdavis.htm",
    "https://www.nwmissouri.edu/naturalsciences/directory/thornsberry.htm",
    "https://www.nwmissouri.edu/admissions/cost/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/biology.htm",
    "https://www.nwmissouri.edu/finaid/apply/fafsa.htm",
    "https://www.nwmissouri.edu/socialsciences/orgs/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/public-administration.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/bas.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/criminology.htm",
    "https://www.nwmissouri.edu/socialsciences/directory/hesse.htm",
    "https://www.nwmissouri.edu/studentaccounts/intlcosts.htm",
    "https://www.nwmissouri.edu/studentaccounts/graduate.htm",
    "https://www.nwmissouri.edu/studentaccounts/specialprograms.htm",
    "https://www.nwmissouri.edu/studentaccounts/roomcosts.htm",
    "https://www.nwmissouri.edu/studentaccounts/online-professional.htm",
    "https://www.nwmissouri.edu/alumni/missing/2005.htm",
    "https://www.nwmissouri.edu/alumni/missing/1970.htm",
    "https://www.nwmissouri.edu/alumni/missing/1990.htm",
    "https://www.nwmissouri.edu/alumni/missing/1960.htm",
    "https://www.nwmissouri.edu/alumni/missing/2015.htm",
    "https://www.nwmissouri.edu/alumni/missing/1965.htm",
    "https://www.nwmissouri.edu/alumni/missing/1975.htm",
    "https://www.nwmissouri.edu/alumni/missing/1980.htm",
    "https://www.nwmissouri.edu/alumni/missing/1985.htm",
    "https://www.nwmissouri.edu/alumni/missing/1995.htm",
    "https://www.nwmissouri.edu/alumni/missing/2000.htm",
    "https://www.nwmissouri.edu/alumni/missing/2010.htm",
    "https://www.nwmissouri.edu/alumni/missing/1950.htm",
    "https://www.nwmissouri.edu/alumni/missing/1940.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/data-science.htm",
    "https://www.nwmissouri.edu/health/contact.htm",
    "https://www.nwmissouri.edu/naturalsciences/about.htm",
    "https://www.nwmissouri.edu/socialsciences/directory/index.htm",
    "https://www.nwmissouri.edu/admissions/cost/bearcatadvantage.htm",
    "https://www.nwmissouri.edu/socialsciences/programs/index.htm",
    "https://www.nwmissouri.edu/socialsciences/missourihope/index.htm",
    "https://www.nwmissouri.edu/media/news/index.htm",
    "https://www.nwmissouri.edu/recreation/how-to-rent.htm",
    "https://www.nwmissouri.edu/recreation/Hours.htm",
    "https://www.nwmissouri.edu/connection/groups/ally.htm",
    "https://www.nwmissouri.edu/hr/Bulletin/2025/October.htm",
    "https://www.nwmissouri.edu/connection/groups/abe.htm",
    "https://www.nwmissouri.edu/hr/employment-opportunities.htm",
    "https://www.nwmissouri.edu/hr/wellness/index.htm",
    "https://www.nwmissouri.edu/connection/groups/community-connections.htm",
    "https://www.nwmissouri.edu/hr/student/trainingopps.htm",
    "https://www.nwmissouri.edu/hr/staff.htm",
    "https://www.nwmissouri.edu/connection/groups/lgbtq.htm",
    "https://www.nwmissouri.edu/hr/trainingopps.htm",
    "https://www.nwmissouri.edu/recreation/HughesFieldhouse/activities.htm",
    "https://www.nwmissouri.edu/recreation/HughesFieldhouse/about.htm",
    "https://www.nwmissouri.edu/housing/current/index.htm",
    "https://www.nwmissouri.edu/finearts/music/yuletide.htm",
    "https://www.nwmissouri.edu/international/contact.htm",
    "https://www.nwmissouri.edu/international/services/index.htm",
    "https://www.nwmissouri.edu/admissions/international/index.htm",
    "https://www.nwmissouri.edu/business/orgs/index.htm",
    "https://www.nwmissouri.edu/business/contact.htm",
    "https://www.nwmissouri.edu/alumni/licenseplate.htm",
    "https://www.nwmissouri.edu/studentaffairs/handbook/index.htm",
    "https://www.nwmissouri.edu/police/emer/index.htm",
    "https://www.nwmissouri.edu/services/ir/consumer/Licensure-Exam.htm",
    "https://www.nwmissouri.edu/finaid/apply/index.htm",
    "https://www.nwmissouri.edu/academics/stateauthorization.htm",
    "https://www.nwmissouri.edu/compserv/copyright-policies.htm",
    "https://www.nwmissouri.edu/finaid/net-price-calculator/index.htm",
    "https://www.nwmissouri.edu/aboutus/facts/heerf-reporting.htm",
    "https://www.nwmissouri.edu/admissions/transfer/joint-admission/index.htm",
    "https://www.nwmissouri.edu/services/ir/consumer/diversity.htm",
    "https://www.nwmissouri.edu/finaid/apply/disbursement.htm",
    "https://www.nwmissouri.edu/hr/eeo.htm",
    "https://www.nwmissouri.edu/career/Post-Grad-Data.htm",
    "https://www.nwmissouri.edu/termsofuse.htm",
    "https://www.nwmissouri.edu/graduate/degreelist.htm",
    "https://www.nwmissouri.edu/accreditation/index.htm",
    "https://www.nwmissouri.edu/finaid/apply/maintaining-eligibility.htm",
    "https://www.nwmissouri.edu/finaid/aid/loans/index.htm",
    "https://www.nwmissouri.edu/academics/stateauthorization-teaching.htm",
    "https://www.nwmissouri.edu/police/sexoffenders.htm",
    "https://www.nwmissouri.edu/finaid/aid/grants.htm",
    "https://www.nwmissouri.edu/services/ir/consumer/retention.htm",
    "https://www.nwmissouri.edu/contact.htm",
    "https://www.nwmissouri.edu/career/students/SARA.htm",
    "https://www.nwmissouri.edu/aboutus/facts/complaintresolution.htm",
    "https://www.nwmissouri.edu/finaid/apply/aid-offer.htm",
    "https://www.nwmissouri.edu/registrar/privacy.htm",
    "https://www.nwmissouri.edu/studentaccounts/billing-and-payments.htm",
    "https://www.nwmissouri.edu/services/ir/consumer/graduation.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/social-science.htm",
    "https://www.nwmissouri.edu/education/contact.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/early-childhood-education.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/elementary-education.htm",
    "https://www.nwmissouri.edu/studentsuccess/support/index.htm",
    "https://www.nwmissouri.edu/academics/online/index.htm",
    "https://www.nwmissouri.edu/CareerReady/index.htm",
    "https://www.nwmissouri.edu/academics/success360/index.htm",
    "https://www.nwmissouri.edu/academics/recovery/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/art.htm",
    "https://www.nwmissouri.edu/finearts/contact.htm",
    "https://www.nwmissouri.edu/media/news/2021/12/magazine/MonicaThornburg.htm",
    "https://www.nwmissouri.edu/ag/mckemy/index.htm",
    "https://www.nwmissouri.edu/ag/agcenter/index.htm",
    "https://www.nwmissouri.edu/giving/opportunities/projects/microcreamy.htm",
    "https://www.nwmissouri.edu/login",
    "https://www.nwmissouri.edu/compserv/tech-help-info.htm",
    "https://www.nwmissouri.edu/getinvolved/cocurricular/index.htm",
    "https://www.nwmissouri.edu/getinvolved/service/index.htm",
    "https://www.nwmissouri.edu/getinvolved/leadership/index.htm",
    "https://www.nwmissouri.edu/wellness/counseling/index.htm",
    "https://www.nwmissouri.edu/wellness/education/calm.htm",
    "https://www.nwmissouri.edu/wellness/services.htm",
    "https://www.nwmissouri.edu/wellness/clinic/index.htm",
    "https://www.nwmissouri.edu/wellness/directory/index.htm",
    "https://www.nwmissouri.edu/wellness/education/index.htm",
    "https://www.nwmissouri.edu/wellness/counseling/tao/index.htm",
    "https://www.nwmissouri.edu/graduate/steps.htm",
    "https://www.nwmissouri.edu/graduate/admissionReqs.htm",
    "https://www.nwmissouri.edu/graduate/requestInfo.htm",
    "https://www.nwmissouri.edu/services/facility/AccessControl.htm",
    "https://www.nwmissouri.edu/services/facility/energy/index.htm",
    "https://www.nwmissouri.edu/services/facility/landscape/index.htm",
    "https://www.nwmissouri.edu/services/facility/workorder.htm",
    "https://www.nwmissouri.edu/services/facility/ehs/index.htm",
    "https://www.nwmissouri.edu/services/facility/capitalprogram/index.htm",
    "https://www.nwmissouri.edu/services/facility/transportation/index.htm",
    "https://www.nwmissouri.edu/services/facility/admin/index.htm",
    "https://www.nwmissouri.edu/services/facility/capitalprogram/signshop/index.htm",
    "https://www.nwmissouri.edu/services/facility/maintenance/central-plant.htm",
    "https://www.nwmissouri.edu/services/facility/maintenance/index.htm",
    "https://www.nwmissouri.edu/services/facility/university-projects.htm",
    "https://www.nwmissouri.edu/services/facility/custodial/index.htm",
    "https://www.nwmissouri.edu/services/facility/vehicle.htm",
    "https://www.nwmissouri.edu/services/sustainability//recycle.htm",
    "https://www.nwmissouri.edu/services/facility/special.htm",
    "https://www.nwmissouri.edu/recreation/FitnessCenter/tour-request.htm",
    "https://www.nwmissouri.edu/recreation/contact.htm",
    "https://www.nwmissouri.edu/recreation/memberships.htm",
    "https://www.nwmissouri.edu/map/trails.htm",
    "https://www.nwmissouri.edu/recreation/FAQ.htm",
    "https://www.nwmissouri.edu/recreation/SportClubs/index.htm",
    "http://www.nwmissouri.edu/registrar/prereg.htm",
    "https://www.nwmissouri.edu/schedule/twbkwbis.P_Logout",
    "https://www.nwmissouri.edu/wtlhelp/twbhhelp.htm",
    "https://www.nwmissouri.edu/fsl/phc/index.htm",
    "https://www.nwmissouri.edu/alumni/events/awards/nominations.htm",
    "https://www.nwmissouri.edu/alumni/events/50th-reunion.htm",
    "https://www.nwmissouri.edu/alumni/events/chipstrong.htm",
    "https://www.nwmissouri.edu/alumni/events/baseballgolf.htm",
    "https://www.nwmissouri.edu/studentaffairs/posting.htm",
    "https://www.nwmissouri.edu/getinvolved/calendarform.htm",
    "https://www.nwmissouri.edu/studentaffairs/absence.htm",
    "https://www.nwmissouri.edu/studentaffairs/policies.htm",
    "https://www.nwmissouri.edu/media/news/2024/12/10collegefactualagranking.htm",
    "https://www.nwmissouri.edu/ag/directory/lacy.htm",
    "https://www.nwmissouri.edu/media/news/2025/06/make-a-difference-as-an-ag-teacher.htm",
    "https://www.nwmissouri.edu/ag/directory/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/ag-science.htm",
    "https://www.nwmissouri.edu/finearts/orgs/index.htm",
    "https://www.nwmissouri.edu/academics/cans/index.htm",
    "https://www.nwmissouri.edu/academics/cbmt/index.htm",
    "https://www.nwmissouri.edu/academics/cehhs/index.htm",
    "https://www.nwmissouri.edu/academics/cacss/index.htm",
    "https://www.nwmissouri.edu/registrar/forms.htm",
    "https://www.nwmissouri.edu/aboutus/mission.htm",
    "https://www.nwmissouri.edu/staff.htm",
    "https://www.nwmissouri.edu/studentsuccess/firstgen/glossary.htm",
    "https://www.nwmissouri.edu/studentsuccess/directory/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/firstgen/staff-resources.htm",
    "https://www.nwmissouri.edu/aboutus/regents/meetings/index.htm",
    "https://www.nwmissouri.edu/rpdc/LETRS.htm",
    "https://www.nwmissouri.edu/rpdc/induction.htm",
    "https://www.nwmissouri.edu/rpdc/summer-trainings.htm",
    "https://www.nwmissouri.edu/rpdc/services/teacher-academy.htm",
    "https://www.nwmissouri.edu/rpdc/networks.htm",
    "https://www.nwmissouri.edu/rpdc/staff.htm",
    "https://www.nwmissouri.edu/knacktive/index.htm",
    "https://www.nwmissouri.edu/business/programs/index.htm",
    "https://www.nwmissouri.edu/media/news/2024/02/16DocuLockfranchise.htm",
    "https://www.nwmissouri.edu/business/nvpc/index.htm",
    "https://www.nwmissouri.edu/business/directory/index.htm",
    "https://www.nwmissouri.edu/map/index.htm",
    "https://www.nwmissouri.edu/compserv/nw-email-scams.htm",
    "https://www.nwmissouri.edu/services/workday/index.htm",
    "https://www.nwmissouri.edu/login/first-time-login.htm",
    "https://www.nwmissouri.edu/career/what-we-do.htm",
    "https://www.nwmissouri.edu/login/catpaws/index.htm",
    "https://www.nwmissouri.edu/studentaffairs/report-incident.htm",
    "https://www.nwmissouri.edu/registrar/prereg.htm",
    "https://www.nwmissouri.edu/compserv/Passwords/index.htm",
    "https://www.nwmissouri.edu/admissions/campusvisits/major.htm",
    "https://www.nwmissouri.edu/admissions/campusvisits/group.htm",
    "https://www.nwmissouri.edu/admissions/campusvisits/transfer.htm",
    "https://www.nwmissouri.edu/admissions/campusvisits/special.htm",
    "https://www.nwmissouri.edu/admissions/campusvisits/saturday.htm",
    "https://www.nwmissouri.edu/parents/health-safety/index.htm",
    "https://www.nwmissouri.edu/parents/planning/index.htm",
    "https://www.nwmissouri.edu/parents/living/index.htm",
    "https://www.nwmissouri.edu/parents/experience.htm",
    "https://www.nwmissouri.edu/parents/learning/index.htm",
    "https://www.nwmissouri.edu/parents/financial/index.htm",
    "https://www.nwmissouri.edu/alumni/travel/interest-survey.htm",
    "https://www.nwmissouri.edu/alumni/travel/past-trips.htm",
    "https://www.nwmissouri.edu/alumni/travel/rome.htm",
    "https://www.nwmissouri.edu/marketing/multimedia/requestform.htm",
    "https://www.nwmissouri.edu/graduation/checklist.htm",
    "https://www.nwmissouri.edu/graduation/dates.htm",
    "https://www.nwmissouri.edu/live/index.htm",
    "https://www.nwmissouri.edu/naturalsciences/contact.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/wildlife-ecology-and-conservation.htm",
    "https://www.nwmissouri.edu/trio/sss/resources.htm",
    "https://www.nwmissouri.edu/admissions/first-year/dualcredit.htm",
    "https://www.nwmissouri.edu/assessment/geneddates.htm",
    "https://www.nwmissouri.edu/communications/directory/index.htm",
    "https://www.nwmissouri.edu/communications/experience/index.htm",
    "https://www.nwmissouri.edu/horacemann/elementary/index.htm",
    "https://www.nwmissouri.edu/horacemann/itc/index.htm",
    "https://www.nwmissouri.edu/horacemann/admissions/tuition.htm",
    "https://www.nwmissouri.edu/horacemann/preschool/index.htm",
    "https://www.nwmissouri.edu/horacemann/faculty/index.htm",
    "https://www.nwmissouri.edu/horacemann/parent/index.htm",
    "https://www.nwmissouri.edu/horacemann/about/index.htm",
    "https://www.nwmissouri.edu/horacemann/admissions/index.htm",
    "https://www.nwmissouri.edu/communications/orgs/index.htm",
    "https://www.nwmissouri.edu/communications/contact.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/child-and-family-studies.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/biology-psychology.htm",
    "https://www.nwmissouri.edu/naturalsciences/orgs/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/human-services.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/english.htm",
    "https://www.nwmissouri.edu/llw/orgs/index.htm",
    "https://www.nwmissouri.edu/llw/contact.htm",
    "https://www.nwmissouri.edu/math/programs/index.htm",
    "https://www.nwmissouri.edu/math/directory/index.htm",
    "https://www.nwmissouri.edu/academics/ltc/quick-access.htm",
    "https://www.nwmissouri.edu/academics/online/teach-for-us.htm",
    "https://www.nwmissouri.edu/academics/ltc/teaching/index.htm",
    "https://www.nwmissouri.edu/academics/ltc/learning/index.htm",
    "https://www.nwmissouri.edu/academics/ltc/learning/showcase/index.htm",
    "https://www.nwmissouri.edu/academics/ltc/teaching/online-best-practices.htm",
    "https://www.nwmissouri.edu/academics/ltc/technology/index.htm",
    "https://www.nwmissouri.edu/academics/ltc/learning/professional-development/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/orientation/fall-start/first-year-student.htm",
    "https://www.nwmissouri.edu/archives/state/roberta/index.htm",
    "https://www.nwmissouri.edu/getinvolved/familyweekend/index.htm",
    "https://www.nwmissouri.edu/archives/state/tower/index.htm",
    "https://www.nwmissouri.edu/getinvolved/homecoming/index.htm",
    "https://www.nwmissouri.edu/archives/university/plaza/index.htm",
    "https://www.nwmissouri.edu/compserv/security-faq.htm",
    "https://www.nwmissouri.edu/compserv/campus-life-tech-policies.htm",
    "https://www.nwmissouri.edu/compserv/mail-harassment-policies.htm",
    "https://www.nwmissouri.edu/policies/technology/index.htm",
    "https://www.nwmissouri.edu/compserv/network-security-policies.htm",
    "https://www.nwmissouri.edu/compserv/report-tech-violations-dp.htm",
    "https://www.nwmissouri.edu/compserv/privacy-policies.htm",
    "https://www.nwmissouri.edu/finearts/directory/index.htm",
    "https://www.nwmissouri.edu/media/news/2024/10/08TowerChoir-NMEA.htm",
    "https://www.nwmissouri.edu/media/news/2025/05/30DauProfessorofInstrumentalMusic.htm",
    "https://www.nwmissouri.edu/finearts/music/scholarships.htm",
    "https://www.nwmissouri.edu/finaid/aid/scholarships/freshman.htm",
    "https://www.nwmissouri.edu/media/news/2023/05/26JaredCarlton.htm",
    "https://www.nwmissouri.edu/registrar/verification.htm",
    "https://www.nwmissouri.edu/textbooks/shipping.htm",
    "https://www.nwmissouri.edu/titleixequity/accessibility/ada-map.htm",
    "https://www.nwmissouri.edu/titleixequity/contact.htm",
    "https://www.nwmissouri.edu/titleixequity/accessibility/accommodations.htm",
    "https://www.nwmissouri.edu/titleixequity/accessibility/parents-family-resources.htm",
    "https://www.nwmissouri.edu/titleixequity/index.htm",
    "https://www.nwmissouri.edu/titleixequity/accessibility/faq.htm",
    "https://www.nwmissouri.edu/titleixequity/accessibility/student-resources.htm",
    "https://www.nwmissouri.edu/titleixequity/accessibility/accessibility-grievance-procedure.htm",
    "https://www.nwmissouri.edu/titleixequity/accessibility/faculty-staff.htm",
    "https://www.nwmissouri.edu/titleixequity/accessibility/forms.htm",
    "https://www.nwmissouri.edu/titleixequity/accessibility/ada-grievance-procedure.htm",
    "https://www.nwmissouri.edu/education/peu/index.htm",
    "https://www.nwmissouri.edu/math/orgs/index.htm",
    "https://www.nwmissouri.edu/math/contact.htm",
    "https://www.nwmissouri.edu/getinvolved/sac/positions.htm",
    "https://www.nwmissouri.edu/getinvolved/sac/contact.htm",
    "https://www.nwmissouri.edu/getinvolved/sac/concerts.htm",
    "https://www.nwmissouri.edu/compserv/new-student-help.htm",
    "https://www.nwmissouri.edu/compserv/printers-copiers.htm",
    "https://www.nwmissouri.edu/compserv/northwest-email.htm",
    "https://www.nwmissouri.edu/compserv/networks-and-connectivity.htm",
    "https://www.nwmissouri.edu/compserv/offices.htm",
    "https://www.nwmissouri.edu/compserv/gaming-entertainment.htm",
    "https://www.nwmissouri.edu/compserv/search.htm",
    "https://www.nwmissouri.edu/studentsuccess/directory/jjohnson.htm",
    "https://www.nwmissouri.edu/fsl/ifc/index.htm",
    "https://www.nwmissouri.edu/services/sustainability/recycle.htm",
    "https://www.nwmissouri.edu/services/sustainability/moveout.htm",
    "https://www.nwmissouri.edu/services/sustainability/northwest.htm",
    "https://www.nwmissouri.edu/services/sustainability/by-the-numbers.htm",
    "https://www.nwmissouri.edu/services/sustainability/contact.htm",
    "https://www.nwmissouri.edu/services/sustainability/recycle-on-campus.htm",
    "https://www.nwmissouri.edu/services/sustainability/studentorganizations.htm",
    "https://www.nwmissouri.edu/services/sustainability/race-to-zero-waste/index.htm",
    "https://www.nwmissouri.edu/admissions/viewbook/index.htm",
    "https://www.nwmissouri.edu/housing/apply/index.htm",
    "https://www.nwmissouri.edu/housing/explore/compare.htm",
    "https://www.nwmissouri.edu/wellness/education/peered/requirements.htm",
    "https://www.nwmissouri.edu/wellness/education/peered/application.htm",
    "https://www.nwmissouri.edu/wellness/education/peered/request.htm",
    "https://www.nwmissouri.edu/studentaffairs/auxiliary/lost-card.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/business-economics.htm",
    "https://www.nwmissouri.edu/admissions/first-year/criteria.htm",
    "https://www.nwmissouri.edu/admissions/first-year/home-schooled.htm",
    "https://www.nwmissouri.edu/admissions/admitted/index.htm",
    "https://www.nwmissouri.edu/admissions/apply/other-applicants/index.htm",
    "https://www.nwmissouri.edu/admissions/transfer/apply.htm",
    "https://www.nwmissouri.edu/naturalsciences/nano/index.htm",
    "https://www.nwmissouri.edu/naturalsciences/biology/index.htm",
    "https://www.nwmissouri.edu/naturalsciences/graduate/index.htm",
    "https://www.nwmissouri.edu/naturalsciences/physics/index.htm",
    "https://www.nwmissouri.edu/naturalsciences/chemistry/index.htm",
    "https://www.nwmissouri.edu/media/news/2023/08/08AdeleShadestudyabroad.htm",
    "https://www.nwmissouri.edu/naturalsciences/geology/index.htm",
    "https://www.nwmissouri.edu/ag/orgs/index.htm",
    "https://www.nwmissouri.edu/media/news/2025/05/how-an-agricultural-business-degree-prepares-you-for-careers.htm",
    "https://www.nwmissouri.edu/familyweekend/family-of-the-year.htm",
    "https://www.nwmissouri.edu/familyweekend/plan-your-visit.htm",
    "https://www.nwmissouri.edu/familyweekend/events.htm",
    "https://www.nwmissouri.edu/fsl/about.htm",
    "https://www.nwmissouri.edu/fsl/stories/index.htm",
    "https://www.nwmissouri.edu/fsl/prospective/index.htm",
    "https://www.nwmissouri.edu/fsl/chapters.htm",
    "https://www.nwmissouri.edu/marketing/web/editing/styles.htm",
    "https://www.nwmissouri.edu/marketing/media/dealingwithmedia.htm",
    "https://www.nwmissouri.edu/marketing/media/welcome.htm",
    "https://www.nwmissouri.edu/marketing/design/logos.htm",
    "https://www.nwmissouri.edu/marketing/events/giveaways.htm",
    "https://www.nwmissouri.edu/marketing/socialmedia/index.htm",
    "https://www.nwmissouri.edu/marketing/events/index.htm",
    "https://www.nwmissouri.edu/media/submit.htm",
    "https://www.nwmissouri.edu/marketing/web/editing/index.htm",
    "https://www.nwmissouri.edu/marketing/media/guide/style/index.htm",
    "https://www.nwmissouri.edu/marketing/design/stationery.htm",
    "https://www.nwmissouri.edu/marketing/design/templates.htm",
    "https://www.nwmissouri.edu/marketing/marketing/index.htm",
    "https://www.nwmissouri.edu/marketing/socialmedia/requestform.htm",
    "https://www.nwmissouri.edu/marketing/design/index.htm",
    "https://www.nwmissouri.edu/aboutus/president/event-request-form.htm",
    "https://www.nwmissouri.edu/marketing/staff.htm",
    "https://www.nwmissouri.edu/marketing/web/index.htm",
    "https://www.nwmissouri.edu/marketing/multimedia/self-service-photos.htm",
    "https://www.nwmissouri.edu/events/user-guide/calendarform.htm",
    "https://www.nwmissouri.edu/marketing/design/requestform.htm",
    "https://www.nwmissouri.edu/marketing/events/requestform.htm",
    "https://www.nwmissouri.edu/housing/current/services/meal.htm",
    "https://www.nwmissouri.edu/academics/dualcredit/student.htm",
    "https://www.nwmissouri.edu/academics/dualcredit/high-school-partner.htm",
    "https://www.nwmissouri.edu/media/news/2024/12/13internetofthingsgreenhouseproject.htm",
    "https://www.nwmissouri.edu/admissions/recruiter/hoyt.htm",
    "https://www.nwmissouri.edu/conferences/Reservations.htm",
    "https://www.nwmissouri.edu/policies/operations/index.htm",
    "https://www.nwmissouri.edu/policies/academics/index.htm",
    "https://www.nwmissouri.edu/policies/student/index.htm",
    "https://www.nwmissouri.edu/policies/faculty/index.htm",
    "https://www.nwmissouri.edu/policies/personnel/index.htm",
    "https://www.nwmissouri.edu/policies/research/index.htm",
    "https://www.nwmissouri.edu/policies/az.htm",
    "https://www.nwmissouri.edu/policies/finance/index.htm",
    "https://www.nwmissouri.edu/assessment/timeline.htm",
    "https://www.nwmissouri.edu/studentsuccess/directory/pieper.htm",
    "https://www.nwmissouri.edu/trio/ubms/about.htm",
    "https://www.nwmissouri.edu/trio/ubms/calendar.htm",
    "https://www.nwmissouri.edu/trio/ubms/apply.htm",
    "https://www.nwmissouri.edu/trio/ubms/forms.htm",
    "https://www.nwmissouri.edu/trio/ubms/scholarships.htm",
    "https://www.nwmissouri.edu/trio/ubms/faqs.htm",
    "https://www.nwmissouri.edu/academics/catalog/ug1214.htm",
    "https://www.nwmissouri.edu/academics/catalog/ug1012.htm",
    "https://www.nwmissouri.edu/academics/catalog/ug0406.htm",
    "https://www.nwmissouri.edu/academics/catalog/ug0204.htm",
    "https://www.nwmissouri.edu/academics/catalog/ug0810.htm",
    "https://www.nwmissouri.edu/academics/catalog/ug0608.htm",
    "https://www.nwmissouri.edu/media/news/2025/11/07MLKcommunityimpactnominations.htm",
    "https://www.nwmissouri.edu/media/news/2025/11/07womenincomputingrecap.htm",
    "https://www.nwmissouri.edu/media/news/2025/11/07strategicplanningkickoo.htm",
    "https://www.nwmissouri.edu/finaid/northwest-promise.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/psychology.htm",
    "https://www.nwmissouri.edu/media/news/2025/11/06Administration-Buildingclosure.htm",
    "https://www.nwmissouri.edu/media/news/2025/11/07poweringdreamsrecap.htm",
    "https://www.nwmissouri.edu/graduate/mba/index.htm",
    "https://www.nwmissouri.edu/marketing/multimedia/videography.htm",
    "https://www.nwmissouri.edu/career/selfie-studio/index.htm",
    "https://www.nwmissouri.edu/education/about.htm",
    "https://www.nwmissouri.edu/education/mcauliffe.htm",
    "https://www.nwmissouri.edu/education/directory/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/recreation.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/biomedical-science-chiropractic.htm",
    "https://www.nwmissouri.edu/studentaffairs/payitforward/request.htm",
    "https://www.nwmissouri.edu/trio/sss/member.htm",
    "https://www.nwmissouri.edu/trio/sss/trio.htm",
    "https://www.nwmissouri.edu/llw/writingcenter/fellowing.htm",
    "https://www.nwmissouri.edu/llw/writingcenter/conversation.htm",
    "https://www.nwmissouri.edu/llw/writingcenter/resources.htm",
    "https://www.nwmissouri.edu/llw/writingcenter/writing.htm",
    "https://www.nwmissouri.edu/llw/writingcenter/contact.htm",
    "https://www.nwmissouri.edu/library/hours.htm",
    "https://www.nwmissouri.edu/library/ask.htm",
    "https://www.nwmissouri.edu/library/Teaching-Resources.htm",
    "https://www.nwmissouri.edu/library/directory.htm",
    "https://www.nwmissouri.edu/library/libraryinfo/study-spaces.htm",
    "https://www.nwmissouri.edu/library/libraryinfo/donation.htm",
    "https://www.nwmissouri.edu/library/libraryinfo/index.htm",
    "https://www.nwmissouri.edu/giving/opportunities/successes/ForeverGreen/index.htm",
    "https://www.nwmissouri.edu/media/news/2021/07/16AASCUpandemicaward.htm",
    "https://www.nwmissouri.edu/admissions/laptop/index.htm",
    "https://www.nwmissouri.edu/media/news/2024/09/25usnewsranking.htm",
    "https://www.nwmissouri.edu/arboretum/arboretumtop50.htm",
    "https://www.nwmissouri.edu/alumni/updateinfo.htm",
    "https://www.nwmissouri.edu/giving/opportunities/index.htm",
    "https://www.nwmissouri.edu/alumni/services.htm",
    "https://www.nwmissouri.edu/alumni/connect.htm",
    "https://www.nwmissouri.edu/alumni/nwretirees/index.htm",
    "https://www.nwmissouri.edu/giving/ways-to-give/index.htm",
    "https://www.nwmissouri.edu/admissions/PDF/Northwest-Admissions.vcf",
    "https://www.nwmissouri.edu/admissions/snapchat.htm",
    "https://www.nwmissouri.edu/admissions/reqform/international.htm",
    "https://www.nwmissouri.edu/admissions/teams.htm",
    "https://www.nwmissouri.edu/admissions/reqform/interestpage.htm",
    "https://www.nwmissouri.edu/admissions/staff.htm",
    "https://www.nwmissouri.edu/admissions/recruiter/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/economics-gis.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/gis.htm",
    "https://www.nwmissouri.edu/socialsciences/directory/bottone.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/marketing-gis.htm",
    "https://www.nwmissouri.edu/csis/orgs/index.htm",
    "https://www.nwmissouri.edu/csis/contact.htm",
    "https://www.nwmissouri.edu/giving/directory/havard.htm",
    "https://www.nwmissouri.edu/giving/directory/untiedt.htm",
    "https://www.nwmissouri.edu/giving/directory/cline.htm",
    "https://www.nwmissouri.edu/giving/directory/jillbrown.htm",
    "https://www.nwmissouri.edu/giving/directory/kessler.htm",
    "https://www.nwmissouri.edu/giving/directory/mjohnson.htm",
    "https://www.nwmissouri.edu/giving/directory/hanson.htm",
    "https://www.nwmissouri.edu/giving/directory/wellhausen.htm",
    "https://www.nwmissouri.edu/giving/directory/long.htm",
    "https://www.nwmissouri.edu/giving/directory/mason.htm",
    "https://www.nwmissouri.edu/giving/directory/schlag.htm",
    "https://www.nwmissouri.edu/giving/directory/dixon.htm",
    "https://www.nwmissouri.edu/international/studyabroad/travel-resources.htm",
    "https://www.nwmissouri.edu/international/studyabroad/preparing.htm",
    "https://www.nwmissouri.edu/international/studyabroad/apply.htm",
    "https://www.nwmissouri.edu/international/studyabroad/directory/index.htm",
    "https://www.nwmissouri.edu/international/studyabroad/funding/index.htm",
    "https://www.nwmissouri.edu/finearts/music/index.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/pre-physical-therapy.htm",
    "https://www.nwmissouri.edu/veterans/needtoknow.htm",
    "https://www.nwmissouri.edu/veterans/inperson.htm",
    "https://www.nwmissouri.edu/veterans/campus.htm",
    "https://www.nwmissouri.edu/admissions/apply/residency.htm",
    "https://www.nwmissouri.edu/education/peu/tess/forms.htm",
    "https://www.nwmissouri.edu/registrar/dynamic-forms.htm",
    "https://www.nwmissouri.edu/studentsuccess/advising/getting-started.htm",
    "https://www.nwmissouri.edu/studentsuccess/coaching/index.htm",
    "https://www.nwmissouri.edu/health/directory/wilmes.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/minors/food-services.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/minors/nutrition.htm",
    "https://www.nwmissouri.edu/registrar/catalogprogramchange.htm",
    "https://www.nwmissouri.edu/health/programs/dietetic-internship/index.htm",
    "https://www.nwmissouri.edu/health/images/programs/Routes-to-RDN-Exam-Eligibility.webp",
    "https://www.nwmissouri.edu/giving/opportunities/projects/martindale/index.htm",
    "https://www.nwmissouri.edu/health/about.htm",
    "https://www.nwmissouri.edu/health/EIM-OC.htm",
    "https://www.nwmissouri.edu/health/directory/index.htm",
    "https://www.nwmissouri.edu/studentaffairs/ssenate/forms/index.htm",
    "https://www.nwmissouri.edu/housing/current/handbook/index.htm",
    "https://www.nwmissouri.edu/socialsciences/directory/gilley.htm",
    "https://www.nwmissouri.edu/archives/nwHistory/index.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/marching/why-join.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/marching/auditions.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/marching/registration.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/marching/events.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/marching/flags.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/marching/pep-band.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/marching/information.htm",
    "https://www.nwmissouri.edu/finearts/directory/strickland.htm",
    "https://www.nwmissouri.edu/finearts/music/ensembles/marching/schedule.htm",
    "https://www.nwmissouri.edu/police/programs/index.htm",
    "https://www.nwmissouri.edu/llw/directory/index.htm",
    "https://www.nwmissouri.edu/giving/giving-societies/index.htm",
    "https://www.nwmissouri.edu/giving/opportunities/scholarships/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/certificates/professional-writing.htm",
    "https://www.nwmissouri.edu/academics/honors/events.htm",
    "https://www.nwmissouri.edu/academics/honors/about.htm",
    "https://www.nwmissouri.edu/academics/honors/benefits.htm",
    "https://www.nwmissouri.edu/academics/honors/eligibility.htm",
    "https://www.nwmissouri.edu/map/tours/dining.htm",
    "https://www.nwmissouri.edu/map/tours/recreation.htm",
    "https://www.nwmissouri.edu/map/tours/athletics.htm",
    "https://www.nwmissouri.edu/map/tours/academics.htm",
    "https://www.nwmissouri.edu/map/tours/reslife.htm",
    "https://www.nwmissouri.edu/events/_data/current.rss",
    "https://www.nwmissouri.edu/events/_data/current-live.json",
    "https://www.nwmissouri.edu/events/_data/current.ics",
    "https://www.nwmissouri.edu/events/_data/current.xml",
    "https://www.nwmissouri.edu/map/construction.htm",
    "https://www.nwmissouri.edu/alert/infrastructure/index.htm",
    "https://www.nwmissouri.edu/recreation/MOERA/maps.htm",
    "https://www.nwmissouri.edu/recreation/MOERA/shootingsports.htm",
    "https://www.nwmissouri.edu/recreation/MOERA/canoe.htm",
    "https://www.nwmissouri.edu/recreation/MOERA/challengecourse.htm",
    "https://www.nwmissouri.edu/hr/student/jobs/jobs.htm",
    "https://www.nwmissouri.edu/hr/student/contactus.htm",
    "https://www.nwmissouri.edu/hr/student/aboutcareerpathing.htm",
    "https://www.nwmissouri.edu/hr/student/faq.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/business-administration.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/certified-supply-chain-professional.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/comptia-cert-training-a-plus-network-plus-security-plus.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/electrical-technician.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/cnc-machinist.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/project-manager-with-capm-pmp-prep.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/automation-technician.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/faq.htm",
    "https://www.nwmissouri.edu/pdcenter/courses/comptia-aplus.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/web-design-professional.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/human-resources-professional.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/medical-billing-and-coding.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/allied-health-professions.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/python-developer.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/data-analyst.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/cmaa-medical-billing-and-coding.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/microsoft-office-master-2019-cert-training.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/engineering-technician.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/paralegal.htm",
    "https://www.nwmissouri.edu/extendedlearning/wioa.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/adobe-certified-associate.htm",
    "https://www.nwmissouri.edu/extendedlearning/cpe/courses/freight-broker-agent-trainer.htm",
    "https://www.nwmissouri.edu/admissions/first-year/index.htm",
    "https://www.nwmissouri.edu/admissions/transfer/index.htm",
    "https://www.nwmissouri.edu/admissions/graduate/index.htm",
    "https://www.nwmissouri.edu/csis/msis/index.htm",
    "https://www.nwmissouri.edu/connection/about.htm",
    "https://www.nwmissouri.edu/connection/resources.htm",
    "https://www.nwmissouri.edu/connection/development/index.htm",
    "https://www.nwmissouri.edu/connection/forms/sponsorshipform.htm",
    "https://www.nwmissouri.edu/connection/development/bearcats-talk.htm",
    "https://www.nwmissouri.edu/connection/directory/index.htm",
    "https://www.nwmissouri.edu/connection/forms/presentationform.htm",
    "https://www.nwmissouri.edu/connection/programs/index.htm",
    "https://www.nwmissouri.edu/connection/student-support/index.htm",
    "https://www.nwmissouri.edu/connection/groups/index.htm",
    "https://www.nwmissouri.edu/finearts/about.htm",
    "https://www.nwmissouri.edu/naturalsciences/directory/gthornsberry.htm",
    "https://www.nwmissouri.edu/ag/contact.htm",
    "https://www.nwmissouri.edu/arboretum/friends/index.htm",
    "https://www.nwmissouri.edu/arboretum/map.htm",
    "https://www.nwmissouri.edu/arboretum/walking-tour.htm",
    "https://www.nwmissouri.edu/homecoming/plan-your-visit.htm",
    "https://www.nwmissouri.edu/media/news/2025/10/07homecomingroyalty.htm",
    "https://www.nwmissouri.edu/events/2025/10/homecoming-open-house.htm",
    "https://www.nwmissouri.edu/events/2025/10/homecoming-bearcat-zone-tailgate.htm",
    "https://www.nwmissouri.edu/homecoming/events.htm",
    "https://www.nwmissouri.edu/homecoming/parade.htm",
    "https://www.nwmissouri.edu/homecoming/registration.htm",
    "https://www.nwmissouri.edu/homecoming/staff.htm",
    "https://www.nwmissouri.edu/archives/computing/albums/ImagesofENIAC/index.htm",
    "https://www.nwmissouri.edu/archives/computing/albums/Jean-ENIACTEAM/index.htm",
    "https://www.nwmissouri.edu/archives/computing/albums/UNIVACI/index.htm",
    "https://www.nwmissouri.edu/archives/computing/albums/Jean-EarlyYears/index.htm",
    "https://www.nwmissouri.edu/archives/computing/albums/Book-Covers/index.htm",
    "https://www.nwmissouri.edu/archives/computing/albums/Electronic-Campus/index.htm",
    "https://www.nwmissouri.edu/archives/computing/albums/Jean-LaterYears/index.htm",
    "https://www.nwmissouri.edu/recreation/FitnessCenter/amenities.htm",
    "https://www.nwmissouri.edu/recreation/FitnessCenter/about.htm",
    "https://www.nwmissouri.edu/communications/about.htm",
    "https://www.nwmissouri.edu/admissions/international/finaid.htm",
    "https://www.nwmissouri.edu/finaid/aid/scholarships/current.htm",
    "https://www.nwmissouri.edu/finaid/aid/scholarships/privresource.htm",
    "https://www.nwmissouri.edu/finaid/aid/scholarships/graduate.htm",
    "https://www.nwmissouri.edu/finaid/aid/scholarships/transfer.htm",
    "https://www.nwmissouri.edu/finaid/aid/index.htm",
    "https://www.nwmissouri.edu/media/social.htm",
    "https://www.nwmissouri.edu/media/experts.htm",
    "https://www.nwmissouri.edu/libraryinfo/index.htm",
    "https://www.nwmissouri.edu/libraryinfo/donation.htm",
    "https://www.nwmissouri.edu/ask.htm",
    "https://www.nwmissouri.edu/Teaching-Resources.htm",
    "https://www.nwmissouri.edu/libraryinfo/study-spaces.htm",
    "https://www.nwmissouri.edu/hours.htm",
    "https://www.nwmissouri.edu/esports/contact.htm",
    "https://www.nwmissouri.edu/esports/request.htm",
    "https://www.nwmissouri.edu/esports/scholarships.htm",
    "https://www.nwmissouri.edu/trio/upwardbound/about.htm",
    "https://www.nwmissouri.edu/trio/upwardbound/requestInfo.htm",
    "https://www.nwmissouri.edu/trio/upwardbound/apply.htm",
    "https://www.nwmissouri.edu/trio/upwardbound/participant.htm",
    "https://www.nwmissouri.edu/trio/upwardbound/forms.htm",
    "https://www.nwmissouri.edu/studentaccounts/costworksheet.htm",
    "https://www.nwmissouri.edu/studentaccounts/paying-your-bill.htm",
    "https://www.nwmissouri.edu/studentaccounts/financialdates.htm",
    "https://www.nwmissouri.edu/finearts/music/media/towerchoir/03_Track_3.mp3",
    "https://www.nwmissouri.edu/finearts/music/media/towerchoir/02_Track_2.mp3",
    "https://www.nwmissouri.edu/finearts/music/media/towerchoir/05_Track_5.mp3",
    "https://www.nwmissouri.edu/finearts/music/media/towerchoir/04_Track_4.mp3",
    "https://www.nwmissouri.edu/finearts/music/media/towerchoir/01_Track_1.mp3",
    "https://www.nwmissouri.edu/academics/undergraduate/minors/agricultural-communications.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/radiological-science.htm",
    "https://www.nwmissouri.edu/services/purchasing/bidlisting.htm",
    "https://www.nwmissouri.edu/services/purchasing/mailcopy/index.htm",
    "https://www.nwmissouri.edu/housing/current/services/student-packages.htm",
    "https://www.nwmissouri.edu/services/purchasing/mailcopy/copyPaperRequestForm.htm",
    "https://www.nwmissouri.edu/services/purchasing/DesignConstruction.htm",
    "https://www.nwmissouri.edu/services/purchasing/contact.htm",
    "https://www.nwmissouri.edu/academics/graduate/masters/biology.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/pre-occupational-therapy.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/minors/environmental-science.htm",
    "https://www.nwmissouri.edu/academics/graduate/masters/teaching-science.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/biology-education.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/physics.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/echocardiographic-sciences.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/physics-education.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/meta-major.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/accel/accelerated-biology.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/marine-biology.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/pre-podiatry.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/certificates/environmental-geology-professional.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/sonographic-sciences.htm",
    "https://www.nwmissouri.edu/media/news/2022/10/05geologysummerfieldtrip.htm",
    "https://www.nwmissouri.edu/registrar/eTranscript.htm",
    "https://www.nwmissouri.edu/intranet/student/index.htm",
    "https://www.nwmissouri.edu/intranet/employee/index.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/information-technology.htm",
    "https://www.nwmissouri.edu/extendedlearning/pd.htm",
    "https://www.nwmissouri.edu/alumni/events/awards/current.htm",
    "https://www.nwmissouri.edu/media/news/2025/10/24TerriVogelMACRAOaward.htm",
    "https://www.nwmissouri.edu/media/news/2025/09/28JohnCoffey500thgame.htm",
    "https://www.nwmissouri.edu/media/news/2025/10/31coldenpondplunge.htm",
    "https://www.nwmissouri.edu/socialsciences/directory/voss.htm",
    "https://www.nwmissouri.edu/media/news/2025/10/31CelebrateAgWeekpreview.htm",
    "https://www.nwmissouri.edu/media/news/2025/11/04WindCymphonyconcertpreview.htm",
    "https://www.nwmissouri.edu/media/news/2025/11/10jazzconcertpreview.htm",
    "https://www.nwmissouri.edu/media/news/spotlights.htm",
    "https://www.nwmissouri.edu/media/news/2025/10/02JennyRyttingJaneAustenConference.htm",
    "https://www.nwmissouri.edu/media/news/2025/10/31mathmovienight.htm",
    "https://www.nwmissouri.edu/naturalsciences/directory/chakraborty.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/emergency-disaster-management.htm",
    "https://www.nwmissouri.edu/registrar/contact.htm",
    "https://www.nwmissouri.edu/alumni/chapters/blackalumnichapter.htm",
    "https://www.nwmissouri.edu/giving/ways-to-give/forms/theatregiving.htm",
    "https://www.nwmissouri.edu/alumni/chapters/blackalumnichaptergiving.htm",
    "https://www.nwmissouri.edu/studentsuccess/support/Complete30/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/orientation/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/first-year-experience/index.htm",
    "https://www.nwmissouri.edu/studentsuccess/retention/index.htm",
    "https://www.nwmissouri.edu/housing/explore/index.htm",
    "https://www.nwmissouri.edu/studentaffairs/auxiliary/station.htm",
    "https://www.nwmissouri.edu/housing/current/get-involved/rha/committees.htm",
    "https://www.nwmissouri.edu/police/directory/index.htm",
    "https://www.nwmissouri.edu/police/contact.htm",
    "https://www.nwmissouri.edu/police/parking/index.htm",
    "https://www.nwmissouri.edu/police/mobileapp.htm",
    "https://www.nwmissouri.edu/csis/about.htm",
    "https://www.nwmissouri.edu/media/news/2022/11/15MadiCombe.htm",
    "https://www.nwmissouri.edu/CyberCenter/index.htm",
    "https://www.nwmissouri.edu/csis/directory/index.htm",
    "https://www.nwmissouri.edu/csis/programs/index.htm",
    "https://www.nwmissouri.edu/media/news/2021/08/23facultyexcellenceawards.htm",
    "https://www.nwmissouri.edu/academics/undergraduate/majors/business-technology.htm",
    "https://www.nwmissouri.edu/alumni/magazine/archives.htm",
    "https://www.nwmissouri.edu/media/news/2025/09/magcoverstory25.htm",
    "https://www.nwmissouri.edu/media/news/2025/06/18CourtneyRowe.htm",
    "https://www.nwmissouri.edu/media/news/2025/06/12KierstenBuckfeature.htm",
    "https://www.nwmissouri.edu/alumni/magazine/current.htm",
    "https://www.nwmissouri.edu/media/news/2025/07/15GoedersEndowedProfessorship.htm",
    "https://www.nwmissouri.edu/media/news/2025/09/magQAMcmenamin.htm",
    "https://www.nwmissouri.edu/media/news/2025/06/24-1975baseballanniversary.htm",
    "https://www.nwmissouri.edu/police/emer/emotional.htm",
    "https://www.nwmissouri.edu/finearts/theatre/index.htm",
    "https://www.nwmissouri.edu/finearts/programs/index.htm",
    "https://www.nwmissouri.edu/finearts/theatre/scholarships.htm",
    "https://www.nwmissouri.edu/finearts/art/index.htm",
    "https://www.nwmissouri.edu/finaid/aid/additional-resources.htm",
    "https://www.nwmissouri.edu/finaid/apply/veterans.htm",
    "https://www.nwmissouri.edu/media/news/2024/05/09militaryfriendly.htm",
    "https://www.nwmissouri.edu/finaid/apply/how-to-apply.htm",
    "https://www.nwmissouri.edu/finaid/contact.htm",
    "https://www.nwmissouri.edu/finaid/cost/index.htm",
    "https://www.nwmissouri.edu/finaid/forms.htm",
    "https://www.nwmissouri.edu/career/events/index.htm",
    "https://www.nwmissouri.edu/career/directory/index.htm",
    "https://www.nwmissouri.edu/career/profession-based-learning.htm",
    "https://www.nwmissouri.edu/career/HABinfo.htm",
    "https://www.nwmissouri.edu/career/staff/index.htm",
    "https://www.nwmissouri.edu/career/students/index.htm",
    "https://www.nwmissouri.edu/career/employers/index.htm",
    "https://www.nwmissouri.edu/career/staff/presentations.htm",
    "https://www.nwmissouri.edu/media/news/2025/06/what-can-you-do-with-a-business-management-degree.htm",
]

# Create knowledge base and store in `vectorstore` variable for reuse
print("Creating knowledge base from web URLs...")
vectorstore = create_knowledge_base(urls, api_key=os.environ.get("PINECONE_API_KEY"), index_name="rag-pipeline")
print('Vectorstore variable is ready for queries.')


Creating knowledge base from web URLs...
Loading content from: https://www.nwmissouri.edu/giving/foundation/board/index.htm
✓ Loaded 1 document(s) from https://www.nwmissouri.edu/giving/foundation/board/index.htm
Loading content from: https://www.nwmissouri.edu/academics/undergraduate/majors/pre-osteopathy.htm
✓ Loaded 1 document(s) from https://www.nwmissouri.edu/academics/undergraduate/majors/pre-osteopathy.htm
Loading content from: https://www.nwmissouri.edu/academics/undergraduate/majors/political-science.htm
✓ Loaded 1 document(s) from https://www.nwmissouri.edu/academics/undergraduate/majors/political-science.htm
Loading content from: https://www.nwmissouri.edu/studentaccounts/tuitionandfees.htm
✓ Loaded 1 document(s) from https://www.nwmissouri.edu/studentaccounts/tuitionandfees.htm
Loading content from: https://www.nwmissouri.edu/alumni/missing/index.htm
✓ Loaded 1 document(s) from https://www.nwmissouri.edu/alumni/missing/index.htm
Loading content from: https://www.nwmissouri.

  0%|          | 0/360 [00:00<?, ?it/s]

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-3-small in organization org-wxN1Uv28tFU1iUgRxSzrVFka on tokens per min (TPM): Limit 1000000, Used 1000000, Requested 63878. Please try again in 3.832s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

## 6) Reusable Query Cell
### **How RAG retrieval works**

1. **Query:** Student enters a question.  
2. **Similarity search:** The `vectorstore.similarity_search(query, k=3)` retrieves the **top 3 most relevant document chunks** from the knowledge base.  
   - `k=3` means we consider **the 3 most similar chunks** for context.  
3. **Augment prompt:** These retrieved chunks are added to the prompt, providing the LLM with relevant context.  
4. **LLM generates answer:** GPT uses the augmented prompt to produce a response.  

### **Key teaching points**

- **Reusable:** This cell can be run **multiple times** for different queries without re-indexing the documents.  
- **Dynamic:** Each query retrieves fresh context from the vectorstore.  
- Students can **experiment with `k`** to see how increasing or decreasing the number of chunks affects answer quality.  


In [6]:
# Example: change this query and re-run as many times as you want
query = "What courses are offered in the MSACS program?"

# Augment with context and call the chat model
augmented_query, sources = augment_prompt(query, vectorstore, k=3)

chat = ChatOpenAI(openai_api_key=os.environ.get("OPENAI_API_KEY"), model="gpt-4o-mini")
system_msg = SystemMessage(content="You are a helpful assistant that answers questions based on the provided context.")

response = chat([system_msg, HumanMessage(content=augmented_query)])
print("\n=== QUESTION ===")
print(query)
print("\n=== ANSWER ===")
print(response.content)
print("\n=== SOURCES ===")
print(", ".join(set(sources)))


/tmp/ipython-input-1331979542.py:10: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat([system_msg, HumanMessage(content=augmented_query)])



=== QUESTION ===
What courses are offered in the MSACS program?

=== ANSWER ===
The MS in Applied Computer Science (MSACS) program offers a variety of computer science courses, which include prerequisite courses and popular electives. Each course is listed with its respective details, including the semester in which it is offered. However, specific course names and details are not provided in the context. For the most accurate and up-to-date information about the courses available, it is recommended to check with the program advisor.

=== SOURCES ===
https://www.nwmissouri.edu/csis/msacs/apply/index.htm, https://www.nwmissouri.edu/csis/msacs/courses.htm, https://www.nwmissouri.edu/csis/msacs/about.htm


## 7) Interactive Chat UI

### **UI Components**

- **Input box:** Where students type their questions.  
- **Buttons:**  
  - **Ask** → sends the query to the RAG Chatbot.  
  - **Clear** → clears the conversation history.  
  - **Quit** → stops the interaction.  
- **Conversation history list:** Stores and displays all previous user queries and GPT answers.  

### **How it works**

1. User enters a question in the input box.  
2. Upon clicking **Ask**, the query is sent through the **retrieval + augmentation process**.  
3. GPT generates an answer using the retrieved context.  
4. The question and answer are displayed dynamically in a **simple chat bubble interface** using HTML.  
5. Buttons allow students to **manage the conversation** interactively.  




In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create widgets
input_box = widgets.Text(placeholder='Type your question here...', description='Question:', layout=widgets.Layout(width='80%'))
ask_button = widgets.Button(description='Ask', button_style='primary')
quit_button = widgets.Button(description='Quit', button_style='danger')
clear_button = widgets.Button(description='Clear', button_style='warning')
output = widgets.HTML(value='', placeholder='', description='')

# Keep conversation history in a list
conversation_history = []

def render_history():
    # Render simple HTML chat window
    html = '<div style="max-height:400px; overflow:auto; border:1px solid #ddd; padding:10px; background:#f9f9f9;">'
    for turn in conversation_history:
        role = turn['role']
        text = turn['text'].replace('\n', '<br>')
        if role == 'user':
            html += f"<div style='text-align:right; margin:6px 0;'><b>You:</b> <span style='background:#dbeafe; padding:6px; border-radius:6px;'>{text}</span></div>"
        else:
            html += f"<div style='text-align:left; margin:6px 0;'><b>Bot:</b> <span style='background:#eef2ff; padding:6px; border-radius:6px;'>{text}</span></div>"
    html += '</div>'
    output.value = html

def on_ask_clicked(b):
    question = input_box.value.strip()
    if not question:
        return
    conversation_history.append({'role':'user','text':question})
    render_history()
    input_box.value = ''

    # Retrieve context and get answer
    augmented_query, sources = augment_prompt(question, vectorstore, k=3)
    chat = ChatOpenAI(openai_api_key=os.environ.get("OPENAI_API_KEY"), model="gpt-4o-mini")
    system_msg = SystemMessage(content="You are a helpful assistant that answers questions based on the provided context.")
    resp = chat([system_msg, HumanMessage(content=augmented_query)])
    answer = resp.content
    conversation_history.append({'role':'bot','text':answer + "<br><br><small><i>Sources: " + ", ".join(set(sources)) + "</i></small>"})
    render_history()

def on_quit_clicked(b):
    clear_output(wait=True)
    print("Chat UI closed. You can re-run the Chat UI cell to reopen it.")

def on_clear_clicked(b):
    conversation_history.clear()
    render_history()

ask_button.on_click(on_ask_clicked)
quit_button.on_click(on_quit_clicked)
clear_button.on_click(on_clear_clicked)

ui = widgets.HBox([input_box, ask_button, clear_button, quit_button])
display(ui, output)
render_history()


HTML(value='', placeholder='')